#### Rating by movie title -> using dataframes on the top of RDD. the latest version of spark support dataframes an is faster than RDD

# Dataframes on the top RDD

# Query two -> step 1: average rating per movie id

# step 1: cache movie dataset in a momory and broadcast

- create two rdd and join them based on key


In [59]:
import collections
import time
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Users\\afoto001.000\\spark'

In [60]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName("SparkApp").setMaster('local')
sc = pyspark.SparkContext(conf=conf) # sc is a spark context that creats an RDD object 
spark = SparkSession(sc)

In [61]:
def parseLine(line):
    fields = line.split(';')
    movie_id = int(fields[1])
    movie_title = float(fields[2])
    return(movie_id,movie_title)
lines = sc.textFile("C:/Users/afoto001.000/Desktop/Abraham_edu_doc/UvA/Master/Big data/BigData/bigdata/data/ml-latest-small/ratings.csv")
rdd = lines.map(parseLine)

In [63]:
totalsById = rdd.mapValues(lambda x:(x,1)).reduceByKey(lambda x,y:(x[0] +y[0],x[1] + y[1]))
averagesById = totalsById.mapValues(lambda x: x[0]/x[1])
results = averagesById.collect()
for result in results:
    print(result)

(1, 3.9209302325581397)
(3, 3.2596153846153846)
(6, 3.946078431372549)
(47, 3.9753694581280787)
(50, 4.237745098039215)
(70, 3.5090909090909093)
(101, 3.782608695652174)
(110, 4.031645569620253)
(151, 3.5454545454545454)
(157, 2.8636363636363638)
(163, 3.5606060606060606)
(216, 3.326530612244898)
(223, 3.855769230769231)
(231, 3.0601503759398496)
(235, 3.6785714285714284)
(260, 4.231075697211155)
(296, 4.197068403908795)
(316, 3.375)
(333, 3.78)
(349, 3.6045454545454545)
(356, 4.164133738601824)
(362, 3.5294117647058822)
(367, 3.1847133757961785)
(423, 2.85)
(441, 3.9285714285714284)
(457, 3.9921052631578946)
(480, 3.75)
(500, 3.388888888888889)
(527, 4.225)
(543, 3.317073170731707)
(552, 3.262295081967213)
(553, 3.8153846153846156)
(590, 3.8353658536585367)
(592, 3.4285714285714284)
(593, 4.161290322580645)
(596, 3.45)
(608, 4.116022099447513)
(648, 3.537037037037037)
(661, 3.4489795918367347)
(673, 2.707547169811321)
(733, 3.6404958677685952)
(736, 3.321138211382114)
(780, 3.44554455

(87869, 3.4375)
(89759, 4.333333333333333)
(93840, 4.0227272727272725)
(96829, 4.3)
(97921, 3.716666666666667)
(100714, 3.5)
(101525, 3.5)
(103980, 3.375)
(111659, 2.75)
(111921, 3.388888888888889)
(112290, 3.823529411764706)
(115569, 4.166666666666667)
(116897, 4.25)
(125916, 1.0)
(128360, 3.75)
(136864, 2.34375)
(122, 2.65625)
(144, 3.3333333333333335)
(227, 2.8636363636363638)
(408, 3.0)
(413, 2.7884615384615383)
(418, 3.0)
(424, 1.8333333333333333)
(433, 2.75)
(436, 2.5714285714285716)
(459, 2.25)
(550, 3.4)
(639, 3.0)
(782, 3.0)
(1191, 3.5)
(1343, 3.78)
(1431, 2.46875)
(1453, 1.8333333333333333)
(1468, 2.6)
(1488, 2.8333333333333335)
(1513, 2.9655172413793105)
(1667, 2.1)
(1726, 2.6923076923076925)
(1810, 3.5)
(1835, 3.3823529411764706)
(1975, 2.5)
(1976, 2.2142857142857144)
(2146, 3.21875)
(2155, 3.625)
(2236, 3.0)
(2297, 3.642857142857143)
(2306, 2.5833333333333335)
(2352, 3.84)
(2404, 1.9166666666666667)
(2433, 3.0454545454545454)
(2497, 2.9166666666666665)
(2500, 2.0)
(2505, 2

(2325, 3.1153846153846154)
(2551, 3.6875)
(2837, 3.0)
(3163, 3.875)
(3325, 2.1666666666666665)
(3719, 3.0)
(3746, 4.0)
(4476, 2.3333333333333335)
(4663, 2.8333333333333335)
(4676, 1.9166666666666667)
(4740, 3.5)
(4912, 3.75)
(4941, 3.1875)
(5460, 3.6666666666666665)
(6012, 2.5)
(6030, 2.0)
(6060, 3.0)
(6169, 2.5)
(6579, 4.0)
(7034, 3.8125)
(7036, 3.142857142857143)
(7050, 3.75)
(7380, 3.357142857142857)
(8643, 3.2777777777777777)
(8796, 3.1)
(25937, 3.3333333333333335)
(32300, 2.8333333333333335)
(45175, 3.75)
(53466, 2.8333333333333335)
(54780, 3.1666666666666665)
(69988, 2.5)
(80584, 4.0)
(85401, 2.9)
(93805, 3.75)
(94015, 3.0)
(94150, 4.0)
(94833, 3.3333333333333335)
(213, 3.8333333333333335)
(665, 4.333333333333333)
(935, 3.8333333333333335)
(1111, 4.125)
(1178, 4.541666666666667)
(1354, 4.071428571428571)
(2131, 4.625)
(2495, 4.0)
(2931, 4.166666666666667)
(3306, 4.125)
(3310, 4.1)
(3592, 4.5)
(3629, 4.071428571428571)
(3677, 4.125)
(3687, 5.0)
(3742, 3.65)
(3821, 2.28125)
(4325, 

(81158, 3.1666666666666665)
(82667, 3.875)
(104272, 3.0)
(110387, 4.0)
(121253, 3.0)
(133115, 4.0)
(773, 3.5)
(823, 3.5)
(1419, 4.25)
(3569, 4.0)
(3736, 3.0)
(3845, 4.0)
(4256, 3.6666666666666665)
(4612, 4.5)
(4708, 4.5)
(4825, 3.0)
(5202, 3.25)
(5238, 3.5)
(5475, 3.8333333333333335)
(5838, 4.0)
(6197, 3.3333333333333335)
(6241, 4.0)
(6329, 3.0)
(7071, 5.0)
(7124, 3.1666666666666665)
(7238, 4.0)
(7459, 4.5)
(7979, 3.5)
(8190, 4.0)
(8484, 4.5)
(8724, 4.25)
(8748, 4.0)
(26003, 4.5)
(26048, 4.5)
(26059, 4.5)
(26073, 5.0)
(26158, 4.0)
(26225, 4.5)
(26317, 3.5)
(26713, 4.5)
(26928, 5.0)
(27003, 3.0)
(27176, 4.5)
(27186, 4.0)
(27328, 4.5)
(31545, 4.0)
(31952, 3.5)
(32511, 4.5)
(32554, 3.8333333333333335)
(32906, 3.5)
(32917, 4.0)
(40955, 3.875)
(42176, 4.0)
(44931, 4.5)
(47446, 3.5)
(48001, 4.0)
(48638, 3.5)
(49917, 4.0)
(51174, 3.5)
(53574, 4.0)
(53883, 4.5)
(54185, 3.75)
(54354, 4.0)
(57772, 3.75)
(58376, 3.75)
(60408, 4.5)
(60885, 3.5)
(61394, 4.0)
(62208, 4.5)
(64167, 3.3333333333333335)

(8755, 3.0)
(8905, 2.5)
(8934, 1.0)
(9004, 2.0)
(26142, 2.0)
(26159, 3.75)
(26176, 2.5)
(26195, 3.0)
(26283, 3.0)
(26285, 2.5)
(26344, 3.0)
(26365, 2.5)
(26413, 3.25)
(26453, 4.0)
(26467, 3.0)
(26510, 2.25)
(26539, 3.0)
(26542, 3.0)
(26547, 4.25)
(26562, 3.0)
(26567, 2.0)
(26590, 2.0)
(26593, 2.0)
(26612, 4.0)
(26622, 3.0)
(26630, 3.0)
(26676, 2.5)
(26696, 0.5)
(26701, 3.0)
(26726, 2.5)
(26741, 1.5)
(26745, 2.5)
(26782, 2.0)
(26797, 3.0)
(26828, 1.5)
(26854, 2.5)
(26861, 3.5)
(26913, 1.5)
(26940, 2.5)
(26947, 2.5)
(27002, 3.5)
(27008, 1.0)
(27032, 3.5)
(27074, 1.5)
(27105, 2.5)
(27124, 2.0)
(27155, 2.0)
(27306, 1.5)
(27420, 2.0)
(27441, 3.25)
(27549, 1.5)
(27595, 0.5)
(27722, 4.166666666666667)
(27762, 2.5)
(27769, 3.0)
(27784, 1.5)
(27829, 3.5)
(31367, 2.0)
(31553, 1.0)
(31664, 3.5)
(31921, 2.5)
(32058, 2.0)
(32440, 2.0)
(32442, 2.5)
(32515, 2.5)
(33126, 2.5)
(33294, 2.5)
(33435, 3.5)
(33629, 2.5)
(33801, 3.0)
(33966, 1.5)
(34292, 1.5)
(34800, 1.5)
(34811, 2.5)
(36850, 3.75)
(39801, 2

In [22]:
type(averagesByAge)

pyspark.rdd.PipelinedRDD

In [58]:
sc.stop()

In [ ]:
ratingById = rating_rdd.mapValues(lambda x:(x,1)).reduceByKey(lambda x,y:(x[0] +y[0],x[1] + y[1]))

In [ ]:
averagesByRating = ratingById.mapValues(lambda x: x[0]/x[1])
results = averagesByRating.collect()
for result in results:
    print(result)

In [ ]:
sc.stop()

In [ ]:
# trail
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
import codecs

def loadMovieNames():
    movieNames = {}
    # CHANGE THIS TO THE PATH TO YOUR u.ITEM FILE:
    with codecs.open("E:/SparkCourse/ml-100k/u.ITEM", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

nameDict = spark.sparkContext.broadcast(loadMovieNames())

# Create schema when reading u.data
schema = StructType([ \
                     StructField("userID", IntegerType(), True), \
                     StructField("movieID", IntegerType(), True), \
                     StructField("rating", IntegerType(), True), \
                     StructField("timestamp", LongType(), True)])

# Load up movie data as dataframe
moviesDF = spark.read.option("sep", "\t").schema(schema).csv("file:///SparkCourse/ml-100k/u.data")

movieCounts = moviesDF.groupBy("movieID").count()

# Create a user-defined function to look up movie names from our broadcasted dictionary
def lookupName(movieID):
    return nameDict.value[movieID]

lookupNameUDF = func.udf(lookupName)

# Add a movieTitle column using our new udf
moviesWithNames = movieCounts.withColumn("movieTitle", lookupNameUDF(func.col("movieID")))

# Sort the results
sortedMoviesWithNames = moviesWithNames.orderBy(func.desc("count"))

# Grab the top 10
sortedMoviesWithNames.show(10, False)

# Stop the session
spark.stop()
